In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import requests as rqs

In [4]:
from bs4 import BeautifulSoup

## Function --- Request on main page

In [5]:
def global_research_scrap(ad,nbr_page):
    """
    ---role
    create a list, for 1 research, of all urls of main page + next pages
    
    ---in 
    nbr_page: number of pages
    ad: root page = 1st page of research
    
    ---out 
    list of main-page urls to scrap
    
    """
    all_url_list = []
    for k in range(2,nbr_page+1):
        adress = ad+"&pi="+str(k)

        page = rqs.get(adress)
        page.status_code

        soup = BeautifulSoup(page.text, "html.parser")

        urlist = soup.find_all("div", class_="column span-three")[1:]
        
        #creation of car's URL lists for each main-page
        for k in range(len(urlist)):
            a = str(urlist[k].find_all("a",class_="listing-title"))[32:270]
            all_url_list.append("https://www.anibis.ch"+a.split("pr=1")[0]+"pr=1")
    print("the number of URLs is:",len(all_url_list))
    return all_url_list

## Function --- Request on individual pages

In [6]:
def _fct_indiv_page(string):
    """
    ---in 
    1 car-level url (string)
    
    ---out 
    1 dictionary of 1 car info.
    
    """
    
    ad_test = string
    page_test = rqs.get(ad_test)
    print(page_test.status_code)
    
    soup_test = BeautifulSoup(page_test.text, "html.parser")
    #print("soup OK")
    
    ##### Price of the car
    c = soup_test.find_all("div", class_="listing-price")
    prix = int(str(c).split("'")[0][-2:]+str(c).split("'")[1][:3])
    #print("prix: ",prix)

    ##### Main information

    b = soup_test.find_all("div",class_="block") 

    #### Create a list with all these fields

    temp_dict = {"description":"NA",
                 "km":"NA",
                 "year":"NA",
                 "boite":"NA",
                 "carburant":"NA",
                 "type":"NA",
                 "chevaux":"NA",
                 "couleur":"NA",
                 "m":"NA",
                 "expertise":"NA",
                 "date_annonce":"NA",
                 "prix":"NA"}
    
    
    for k in range(len(b)):
        #print(k)
        if "Modèle" in str(b[k]):
            temp_dict["description"]=b[k].contents[1]
        elif "Kilomètres" in str(b[k]):
            if len(b[k].contents[1])==5:
                temp_dict["km"] = float(b[3].contents[1][:-3])
            elif len(b[k])==2:
                temp_dict["km"] =str(b[k]).split("km")[0][-2:-1]
            else:
                temp_dict["km"] = float(str(b[3].contents[1]).split("'")[0]+str(b[3].contents[1]).split("'")[1][:-3])
        elif "Année" in str(b[k]):
            temp_dict["year"]=int(b[k].contents[1])
        elif "Transmission" in str(b[k]):
            temp_dict["boite"]=b[k].contents[1][:-1]
        elif "Carburant" in str(b[k]):
            temp_dict["carburant"]=b[k].contents[1][:-1]
        elif "Carrosserie" in str(b[k]):
            temp_dict["type"]=b[k].contents[1][:-1]
        elif "Puissance" in str(b[k]):
            temp_dict["chevaux"]=int(b[k].contents[1][:-1])
        elif "Couleur" in str(b[k]):
            temp_dict["couleur"]=b[k].contents[1][:-1]
        elif "motrices" in str(b[k]):
            temp_dict["m"]=b[k].contents[1][:-1]
        elif "Expertisé" in str(b[k]):
            if str(b[k])=="✓":
                temp_dict["expertise"]=1
            else:
                temp_dict["expertise"]=0
        elif "Actualisé" in str(b[k]):
            temp_dict["date_annonce"] = str(b[13])[84:94]
    temp_dict["prix"]=prix
    
    
    return pd.Series(temp_dict)

In [7]:
def old_fct_indiv_page(string):
    """
    ---in 
    1 car-level url (string)
    
    ---out 
    1 dictionary of 1 car info.
    
    """
    
    ad_test = string
    page_test = rqs.get(ad_test)
    print(page_test.status_code)
    
    soup_test = BeautifulSoup(page_test.text, "html.parser")
    #print("soup OK")
    
    ##### Price of the car
    c = soup_test.find_all("div", class_="listing-price")
    prix = int(str(c).split("'")[0][-2:]+str(c).split("'")[1][:3])
    print("prix: ",prix)

    ##### Main information

    b = soup_test.find_all("div",class_="block") 

    #### Create a list with all these fields

    temp_dict = {"description":"NA",
                 "km":"NA",
                 "year":"NA",
                 "boite":"NA",
                 "carburant":"NA",
                 "type":"NA",
                 "chevaux":"NA",
                 "couleur":"NA",
                 "m":"NA",
                 "expertise":"NA",
                 "date_annonce":"NA",
                 "prix":"NA"}
    
    
    for k in range(len(b)):
        #print(k)
        if "Modèle" in str(b[k]):
            temp_dict["description"]=b[k].contents[1]
        elif "Kilomètres" in str(b[k]):
            if len(b[k].contents[1])==5:
                temp_dict["km"] = float(b[3].contents[1][:-3])
            else:
                temp_dict["km"] = float(str(b[3].contents[1]).split("'")[0]+str(b[3].contents[1]).split("'")[1][:-3])
        elif "Année" in str(b[k]):
            temp_dict["year"]=int(b[k].contents[1])
        elif "Transmission" in str(b[k]):
            temp_dict["boite"]=b[k].contents[1][:-1]
        elif "Carburant" in str(b[k]):
            temp_dict["carburant"]=b[k].contents[1][:-1]
        elif "Carrosserie" in str(b[k]):
            temp_dict["type"]=b[k].contents[1][:-1]
        elif "Puissance" in str(b[k]):
            temp_dict["chevaux"]=int(b[k].contents[1][:-1])
        elif "Couleur" in str(b[k]):
            temp_dict["couleur"]=b[k].contents[1][:-1]
        elif "motrices" in str(b[k]):
            temp_dict["m"]=b[k].contents[1][:-1]
        elif "Expertisé" in str(b[k]):
            if str(b[k])=="✓":
                temp_dict["expertise"]=1
            else:
                temp_dict["expertise"]=0
        elif "Actualisé" in str(b[k]):
            temp_dict["date_annonce"] = str(b[13])[84:94]
    temp_dict["prix"]=prix
    
    
    return pd.Series(temp_dict)

### Function - integrate all data in dataframe

In [8]:
def single_series_to_df(list_of_series):
    """
    ---in 
    list of series out of _fct_indiv_page
    
    ---out 
    Full dataframe with all data
    
    """
    df = pd.DataFrame()
    for k in range(len(list_of_series)):
            df = df.append(list_of_series[k],ignore_index=True)
        
    return df

### Function --- All In One Function

In [12]:
def fct_all_in_one(ad,nbr_page):
    all_urls = global_research_scrap(ad,nbr_page)
    
    final_list = []
    
    for k in range(len(all_urls)):
        print(k)
        final_list.append(_fct_indiv_page(all_urls[k]))
    return single_series_to_df(final_list)

# Execution

##### version function

In [10]:
#parameters
ad = "https://www.anibis.ch/fr/automobiles-voitures-de-tourisme--113/advertlist.aspx?fts=skoda+octavia&atxl=13358_%25octavia%25&aral=834_15000_50000&aidl=103,15222&dlf=2"
nbr_page = 100

In [13]:
out = fct_all_in_one(ad,nbr_page)
out

the number of URLs is: 1940
0
200
1
200
2
200
3
200
4
200
5
200
6
200
7
200
8
200
9
200
10
200
11
200
12
200
13
200
14
200
15
200
16
200
17
200
18
200
19
200
20
200
21
200
22
200
23
200
24
200
25
200
26
200
27
200
28
200
29
200
30
200
31
200
32
200
33
200
34
200
35
200
36
200
37
200
38
200
39
200
40
200
41
200
42
200
43
200
44
200
45
200
46
200
47
200
48
200
49
200
50
200
51
200
52
200
53
200
54
200
55
200
56
200
57
200
58
200
59
200
60
200
61
200
62
200
63
200
64
200
65
200
66
200
67
200
68
200
69
200
70
200
71
200
72
200
73
200
74
200
75
200
76
200
77
200
78
200
79
200
80
200
81
200
82
200
83
200
84
200
85
200
86
200
87
200
88
200
89
200
90
200
91
200
92
200
93
200
94
200
95
200
96
200
97
200
98
200
99
200
100
200
101
200
102
200
103
200
104
200
105
200
106
200
107
200
108
200
109
200
110
200
111
200
112
200
113
200
114
200
115
200
116
200
117
200
118
200
119
200
120
200
121
200
122
200
123
200
124
200
125
200
126
200
127
200
128
200
129
200
130
200
131
200
132
200
133
200
134
200
13

IndexError: list index out of range

In [17]:
out

NameError: name 'out' is not defined

out.to_csv("first_list.csv")

##### version by hand

In [15]:
#parameters
ad = "https://www.anibis.ch/fr/automobiles-voitures-de-tourisme--113/advertlist.aspx?fts=skoda+octavia&atxl=13358_%25octavia%25&aral=834_15000_50000&aidl=103,15222&dlf=2"
nbr_page = 100

In [16]:
%%time
all_urls = global_research_scrap(ad,nbr_page)

the number of URLs is: 1940
Wall time: 1min 45s


In [18]:
all_urls[499]

"https://www.anibis.ch/fr/d-automobiles-voitures-de-tourisme--113/skoda-octavia-rs,-2017,-31'000-km--28876856.aspx?fts=skoda+octavia&amp;atxl=13358_%25octavia%25&amp;aral=834_15000_50000&amp;aidl=103,15222&amp;dlf=2&amp;view=2&amp;pi=26&amp;fcid=113&amp;pr=1"

In [19]:
len(all_urls)

1940

In [21]:
%%time
final_list = []
for k in range(len(all_urls)):
    print(k)
    final_list.append(_fct_indiv_page(all_urls[k]))


0
200
1
200
2
200
3
200
4
200
5
200
6
200
7
200
8
200
9
200
10
200
11
200
12
200
13
200
14
200
15
200
16
200
17
200
18
200
19
200
20
200
21
200
22
200
23
200
24
200
25
200
26
200
27
200
28
200
29
200
30
200
31
200
32
200
33
200
34
200
35
200
36
200
37
200
38
200
39
200
40
200
41
200
42
200
43
200
44
200
45
200
46
200
47
200
48
200
49
200
50
200
51
200
52
200
53
200
54
200
55
200
56
200
57
200
58
200
59
200
60
200
61
200
62
200
63
200
64
200
65
200
66
200
67
200
68
200
69
200
70
200
71
200
72
200
73
200
74
200
75
200
76
200
77
200
78
200
79
200
80
200
81
200
82
200
83
200
84
200
85
200
86
200
87
200
88
200
89
200
90
200
91
200
92
200
93
200
94
200
95
200
96
200
97
200
98
200
99
200
100
200
101
200
102
200
103
200
104
200
105
200
106
200
107
200
108
200
109
200
110
200
111
200
112
200
113
200
114
200
115
200
116
200
117
200
118
200
119
200
120
200
121
200
122
200
123
200
124
200
125
200
126
200
127
200
128
200
129
200
130
200
131
200
132
200
133
200
134
200
135
200
136
200
137
200
138
20

IndexError: list index out of range

In [24]:
out = single_series_to_df(final_list)

In [25]:
out

,boite,carburant,chevaux,couleur,date_annonce,description,expertise,km,m,prix,type,year
0,Automatique,Essence,220,Gris,ingLocatio,Octavia Combi 2.0 TSI RS DSG,NA,0,Traction avant,19000.0,Break,2014.0
1,Automatique,Essence,180,Gris,ingLocatio,Octavia 1.8TSI Scout 4x4,NA,0,4 roues motrices,19950.0,Break,2015.0
2,Manuelle,Diesel,NA,NA,"{""event"":""",Octavia,0,0,Traction avant,18350.0,Break,2017.0
3,Manuelle,Diesel,184,Gris,ingLocatio,Octavia Combi 2.0 TDI RS,NA,0,Traction avant,22900.0,Break,2015.0
4,Automatique,Essence,180,Noir,01.06.2019,Octavia Combi 1.8 TSI Ambition 4x4 DSG,0,0,4 roues motrices,19600.0,Break,2016.0
5,Automatique,Essence,230,Gris,01.06.2019,Octavia 2.0 TSI RS 230,0,0,Traction avant,32900.0,Break,2015.0
6,Automatique,Diesel,184,Argent,01.06.2019,Octavia Combi 2.0 TDI Style 4x4,0,3,4 roues motrices,27500.0,Break,2018.0
7,Manuelle,Diesel,115,Argent,01.06.2019,OCTAVIA Combi 1.6 TDI 115 Ambition 4x4,0,30,4 roues motrices,27560.0,Break,2019.0
8,Manuelle,Diesel,184,Noir,01.06.2019,Octavia 2.0 TDI RS,0,0,Traction avant,17820.0,Break,2013.0
9,Automatique,"Gas (LNG, LPG, GPL, CNG)",110,Blanc,01.06.2019,Octavia Combi 1.4 TSI G-tec Ambition CNG DSG,0,0,Traction avant,22800.0,Break,2018.0


In [26]:
out.to_csv("list_v500.csv")

# ARCHIVES - BEFORE FUNCTIONS

## Request on main page

adress = "https://www.anibis.ch/fr/automobiles-voitures-de-tourisme--113/advertlist.aspx?fts=skoda+octavia&atxl=13358_%25octavia%25&aral=834_15000_50000&aidl=103,15222&dlf=1"

page = rqs.get(adress)
page.status_code

page.text[:400]

soup = BeautifulSoup(page.text, "html.parser")

urlist = soup.find_all("div", class_="column span-three")[1:]

g_urlist = []
for k in range(len(urlist)):
    a = str(urlist[k].find_all("a",class_="listing-title"))[32:270]
    g_urlist.append("https://www.anibis.ch"+a.split("pr=1")[0]+"pr=1")
    #print(g_urlist[k])

g_urlist[2]

## Request on individual pages

ad_test = "https://www.anibis.ch/fr/d-automobiles-voitures-de-tourisme--113/skoda-octavia,-2017,-16'700-km--29305272.aspx?fts=skoda+octavia&amp;atxl=13358_%25octavia%25&amp;aral=834_15000_50000&amp;aidl=103,15222&amp;dlf=2&amp;view=2&amp;pi=2&amp;fcid=113&amp;pr=1"
ad_test = "https://www.anibis.ch/fr/d-automobiles-voitures-de-tourisme--113/skoda-octavia-combi-20-tsi-rs-245-dsg--29310916.aspx?fts=skoda+octavia&amp;atxl=13358_%25octavia%25&amp;aral=834_15000_50000&amp;aidl=103,15222&amp;dlf=1&amp;view=2&amp;fcid=113&amp;pr=1"
#ad_test = "https://www.anibis.ch/fr/d-automobiles-voitures-de-tourisme--113/skoda-octavia-combi-20-tdi-rs--29302350.aspx?fts=skoda+octavia&amp;atxl=13358_%25octavia%25&amp;aral=834_15000_50000&amp;aidl=103,15222&amp;dlf=2&amp;view=2&amp;pi=2&amp;fcid=113&amp;pr=1"
ad_test = "https://www.anibis.ch/fr/d-automobiles-voitures-de-tourisme--113/skoda-octavia-combi-20-tsi-rs-245-dsg--29310916.aspx?fts=skoda+octavia&amp;atxl=13358_%25octavia%25&amp;aral=834_15000_50000&amp;aidl=103,15222&amp;dlf=1&amp;view=2&amp;fcid=113&amp;pr=1"

page_test = rqs.get(ad_test)
page_test.status_code

soup_test = BeautifulSoup(page_test.text, "html.parser")

##### Price of the car

c = soup_test.find_all("div", class_="listing-price")

prix = str(c)[37:43]

prix

##### Main information

b = soup_test.find_all("div",class_="block") 

#### Create a list with all these fields

if "Kilomètres" in str(b[3]):
    print("ok")

len(b)

str(b[13])[84:94]

temp_dict = {"description":"NA",
 "km":"NA",
 "year":"NA",
 "boite":"NA",
 "carburant":"NA",
 "type":"NA",
 "chevaux":"NA",
 "couleur":"NA",
 "m":"NA",
 "expertise":"NA",
 "prix":"NA"}

%%time
for k in range(len(b)):
    #print(k)
    if "Modèle" in str(b[k]):
        temp_dict["description"]=b[k].contents[1]
    elif "Kilomètres" in str(b[k]):
        if len(b[k].contents[1])==5:
            temp_dict["km"] = float(b[3].contents[1][:-3])
        else:
            temp_dict["km"] = float(str(b[3].contents[1]).split("'")[0]+str(b[3].contents[1]).split("'")[1][:-3])
    elif "Année" in str(b[k]):
        temp_dict["year"]=int(b[k].contents[1])
    elif "Transmission" in str(b[k]):
        temp_dict["boite"]=b[k].contents[1][:-1]
    elif "Carburant" in str(b[k]):
        temp_dict["carburant"]=b[k].contents[1][:-1]
    elif "Carrosserie" in str(b[k]):
        temp_dict["type"]=b[k].contents[1][:-1]
    elif "Puissance" in str(b[k]):
        temp_dict["chevaux"]=int(b[k].contents[1][:-1])
    elif "Couleur" in str(b[k]):
        temp_dict["couleur"]=b[k].contents[1][:-1]
    elif "motrices" in str(b[k]):
        temp_dict["m"]=b[k].contents[1][:-1]
    elif "Expertisé" in str(b[k]):
        if str(b[k])=="✓":
            temp_dict["expertise"]=1
        else:
            temp_dict["expertise"]=0
temp_dict["prix"]=int(prix[:2]+prix[3:7])

temp_dict

attr_car

col_name = ["description","km","year","boite","carburant","type","chevaux","couleur","m","contr_tech","prix"]

row = pd.Series(attr_car,col_name)

row

## Integrate all data in dataframe

g_urlist[4]

df = pd.DataFrame()

for k in range(len(g_urlist)):
    row = fct_indiv_page(g_urlist[k])
    df = df.append([row],ignore_index=True)

df1 = df

g_urlist[2]